In [5]:
from bacp import BaCPTrainer, BaCPTrainingArguments
from trainer import Trainer, TrainingArguments
from utils import print_statistics, get_device, get_num_workers, print_dynamic_lambdas_statistics, save_object, load_object
from constants import *

In [6]:
MODEL_NAME = 'resnet50'
MODEL_TASK = 'cifar10'
FINETUNED_WEIGHTS = './research/resnet50/cifar10/resnet50_cifar10_baseline.pt'
TRAIN = False
DEVICE = get_device()
NUM_WORKERS = get_num_workers()
print("Using device:", DEVICE)
print("Using", NUM_WORKERS, "workers")

Using device: cuda
Using 288 workers


## Baseline Check

In [7]:
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    scheduler_type='linear_with_warmup',
    epochs=100,
    learning_type="baseline",
    patience=50,
    db=False,
)
trainer = Trainer(training_args=training_args)
metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Linear scheduler initialized with warmup steps: 830 and total steps: 8300
[TRAINER] Pruning not initialized
[TRAINER] Saving model to: ./research/resnet50/cifar10/resnet50_cifar10_baseline.pt
[TRAINER] Loading weights: ./research/resnet50/cifar10/resnet50_cifar10_baseline.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.0



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     93.02%

Model Information:
------------------------------
  Total Parameters:     23,520,842
  Trainable Parameters: 23,520,842

Training Configuration:
------------------------------
  Model:                resnet50
  Task:                 cifar10
  Learning Type:        baseline
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               100

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## BaCP

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

all_metrics = {}
for i in range(4):
    lambdas = [1]*4
    lambdas[i] = 0
    types = ['no_ce', 'no_prc', 'no_snc', 'no_fic']

    bacp_training_args = BaCPTrainingArguments(
        model_name=MODEL_NAME,
        model_task=MODEL_TASK,
        batch_size=BATCH_SIZE_CNN,
        optimizer_type_and_lr=('sgd', 0.1),
        pruning_type='magnitude_pruning',
        target_sparsity=TARGET_SPARSITY_LOW,
        sparsity_scheduler='cubic',
        finetuned_weights=finetuned_weights,
        learning_type='ablation_b2_' + types[i],
        db=False,
        log_epochs=False,
    )

    bacp_trainer = BaCPTrainer(bacp_training_args, lambdas=lambdas)
    bacp_trainer.train()

    bacp_trainer.generate_mask_from_model()
    training_args = TrainingArguments(
        model_name=bacp_trainer.model_name,
        model_task=bacp_trainer.model_task,
        batch_size=bacp_trainer.batch_size,
        optimizer_type_and_lr=('adamw', 0.0001),
        pruner=bacp_trainer.get_pruner(),
        pruning_type=bacp_trainer.pruning_type,
        target_sparsity=bacp_trainer.target_sparsity,
        epochs=50,
        finetuned_weights=bacp_trainer.save_path,
        finetune=True,
        learning_type=f"ablation_b2_{types[i]}_finetune",
        db=False,
        log_epochs=False,
    )
    trainer = Trainer(training_args)
    if True:
        trainer.train()

    metrics = trainer.evaluate()
    print_statistics(metrics, trainer)

    all_metrics[types[i]] = metrics

save_object(all_metrics, f'{MODEL_NAME}_{MODEL_TASK}_{bacp_trainer.pruning_type}_{bacp_trainer.target_sparsity}_b2_metrics.pkl')

[TRAINER] Image size: 32
[ERROR] Could not load weights: ./research/resnet50/cifar10/resnet50_cifar10_baseline.pt
[ERROR] Attempting partial load
[TRAINER] Weights loaded successfully
[TRAINER] Initialized BaCP models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.1)
[TRAINER] No scheduler initialized
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/resnet50/cifar10/resnet50_cifar10_magnitude_pruning_0.95_ablation_b2_no_ce.pt


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Epoch [1/5]: Avg Total Loss: 9.6899 | Avg PrC Loss: 4.8300 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 4.8599 | Avg CE Loss: 0.0000 | Model Sparsity: 0.4636

[BaCP] weights saved!


Retraining Epoch [1/10]: Avg Total Loss: 9.2093 | Avg PrC Loss: 4.7111 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 4.4982 | Avg CE Loss: 0.0000 | Model Sparsity: 0.4636



...

Training epoch [25/50]: Avg Loss: 0.0608 | Avg Accuracy: 95.02 | Model Sparsity: 0.95



Training epoch [26/50]: Avg Loss: 0.0577 | Avg Accuracy: 95.01 | Model Sparsity: 0.95



Training epoch [27/50]: Avg Loss: 0.0545 | Avg Accuracy: 95.17 | Model Sparsity: 0.95



Training epoch [28/50]: Avg Loss: 0.0543 | Avg Accuracy: 95.12 | Model Sparsity: 0.95



Training Epoch [29/50]:  80%|███████▉  | 66/83 [00:05<00:01, 10.88it/s, Loss=0.0516, Sparsity=0.95]

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

all_metrics = {}
for i in range(4):
    lambdas = [1]*4
    lambdas[i] = 0
    types = ['no_ce', 'no_prc', 'no_snc', 'no_fic']

    bacp_training_args = BaCPTrainingArguments(
        model_name=MODEL_NAME,
        model_task=MODEL_TASK,
        batch_size=BATCH_SIZE_CNN,
        optimizer_type_and_lr=('sgd', 0.1),
        pruning_type='magnitude_pruning',
        target_sparsity=TARGET_SPARSITY_MID,
        sparsity_scheduler='cubic',
        finetuned_weights=finetuned_weights,
        learning_type='ablation_b2_' + types[i],
        db=False,
        log_epochs=False,
    )

    bacp_trainer = BaCPTrainer(bacp_training_args, lambdas=lambdas)
    bacp_trainer.train()

    bacp_trainer.generate_mask_from_model()
    training_args = TrainingArguments(
        model_name=bacp_trainer.model_name,
        model_task=bacp_trainer.model_task,
        batch_size=bacp_trainer.batch_size,
        optimizer_type_and_lr=('adamw', 0.0001),
        pruner=bacp_trainer.get_pruner(),
        pruning_type=bacp_trainer.pruning_type,
        target_sparsity=bacp_trainer.target_sparsity,
        epochs=50,
        finetuned_weights=bacp_trainer.save_path,
        finetune=True,
        learning_type=f"ablation_b2_{types[i]}_finetune",
        db=False,
        log_epochs=False,
    )
    trainer = Trainer(training_args)
    if True:
        trainer.train()

    metrics = trainer.evaluate()
    print_statistics(metrics, trainer)


    all_metrics[types[i]] = metrics

save_object(all_metrics, f'{MODEL_NAME}_{MODEL_TASK}_{bacp_trainer.pruning_type}_{bacp_trainer.target_sparsity}_b2_metrics.pkl')

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

all_metrics = {}
for i in range(4):
    lambdas = [1]*4
    lambdas[i] = 0
    types = ['no_ce', 'no_prc', 'no_snc', 'no_fic']

    bacp_training_args = BaCPTrainingArguments(
        model_name=MODEL_NAME,
        model_task=MODEL_TASK,
        batch_size=BATCH_SIZE_CNN,
        optimizer_type_and_lr=('sgd', 0.1),
        pruning_type='magnitude_pruning',
        target_sparsity=TARGET_SPARSITY_HIGH,
        sparsity_scheduler='cubic',
        finetuned_weights=finetuned_weights,
        learning_type='ablation_b2_' + types[i],
        db=False,
        log_epochs=False,
    )

    bacp_trainer = BaCPTrainer(bacp_training_args, lambdas=lambdas)
    bacp_trainer.train()

    bacp_trainer.generate_mask_from_model()
    training_args = TrainingArguments(
        model_name=bacp_trainer.model_name,
        model_task=bacp_trainer.model_task,
        batch_size=bacp_trainer.batch_size,
        optimizer_type_and_lr=('adamw', 0.0001),
        pruner=bacp_trainer.get_pruner(),
        pruning_type=bacp_trainer.pruning_type,
        target_sparsity=bacp_trainer.target_sparsity,
        epochs=50,
        finetuned_weights=bacp_trainer.save_path,
        finetune=True,
        learning_type=f"ablation_b2_{types[i]}_finetune",
        db=False,
        log_epochs=False,
    )
    trainer = Trainer(training_args)
    if True:
        trainer.train()

    metrics = trainer.evaluate()
    print_statistics(metrics, trainer)


    all_metrics[types[i]] = metrics

save_object(all_metrics, f'{MODEL_NAME}_{MODEL_TASK}_{bacp_trainer.pruning_type}_{bacp_trainer.target_sparsity}_b2_metrics.pkl')